# Table of contents

[Table 1](#table1) <br/>
[Table S1](#tables1) <br/>
[Table S2](#tables2) <br/>
[Table S3](#tables3) <br/>
[Table S4](#tables4) <br/>
[Table S5](#tables5) <br/>
[Table S6](#tables6) <br/>
[Table S7](#tables7) <br/>
[Table S8](#tables8) <br/>

In [1]:
# # # recreate the exact environment

# conda create -y -n morphine_cfos_paper_jupyter python=3.9
# conda activate morphine_cfos_paper_jupyter
# pip install notebook==6.4.10
# pip install bg-atlasapi==1.0.2
# pip install brainrender==2.1.9
# pip install statannotations==0.6.0
# pip install seaborn==0.12.2
# pip install plotly==5.24.1
# pip install jupyterlab==3.3.3
# pip install traitlets==5.9.0
# pip install --upgrade pandas==1.4.2
# pip install --upgrade numpy==1.24
# pip install --upgrade matplotlib==3.5.1
# pip install kaleido==0.2.1
# # pip install --upgrade importlib-metadata==6.8.0  # if there's a metadata related error

In [2]:
import os
import json
from collections import Counter, defaultdict
from glob import glob

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
from IPython.display import display, HTML
from bg_atlasapi.bg_atlas import BrainGlobeAtlas
# from brainrender import Scene
# from brainrender._colors import map_color
from datetime import datetime
from scipy import stats

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', 100)

In [3]:
start_time = datetime.now()  # to measure how long the whole notebook takes to run

In [4]:
# constants
significant_only = True
significance_threshold = 0.05
volume_mm3_threshold = 0.25
exclude_fiber_tracts = True
min_cells = 25

large_structures = ['Isocortex', 'CTXsp', 'HPF', 'OLF', 'HY', 'MB', 'P', 'PAL', 'STR', 'TH', 'MY', 'CB']


In [5]:
# atlas related

atlas = BrainGlobeAtlas("allen_mouse_10um")
tree = atlas.structures.tree
root_node_id = tree.root
get_acronym = lambda x: x.split(">")[-1]

# "csv_files" archive needs to be downloaded and unpacked (see README.md file for instructions)
region_volumes = json.load(open("csv_files/region_volumes_10um.json", "r"))

def structure_from_acronym(acronym):
    return ">".join(atlas.get_structure_ancestors(acronym) + [acronym])

def exclude_parents(acronyms):
    """
    Goes through the list of acronyms, and removes structures that are parents of other structures in this list.
    For example, if list contains both ACB and STR (STR being parent of ACB), this function will exclude STR
    """
    acronyms_filtered = acronyms[:]
    for acronym in acronyms:
        parents = atlas.get_structure_ancestors(acronym)
        for parent in parents:
            if parent in acronyms_filtered:
                acronyms_filtered.remove(parent)
    return acronyms_filtered

def get_major_parent(acronym):
    parents = atlas.get_structure_ancestors(acronym)
    for structure in large_structures:
        if structure in parents:
            return structure
    return "Other"

In [6]:
# filtering functions
def is_large_enough(acronym):
    volume = region_volumes[acronym]
    return volume >= volume_mm3_threshold

def is_grey(acronym):
    parents = atlas.get_structure_ancestors(acronym)
    return "grey" in parents

def both_above_threshold(structure, df1, df2):
    return (
        df1[df1['acronym'] == structure]['cell_count'].values[0] >= min_cells 
        and df2[df2['acronym'] == structure]['cell_count'].values[0] >= min_cells 
    )

In [7]:
# calculate p-values between two datasets using unpaired t-test
def get_p_value(ds1, ds2):
    t_stat, p_val = stats.ttest_ind(ds1, ds2)
    return p_val

In [8]:
def get_all_structures_cell_density(df_path):
    print("reading DF:", df_path)
    raw_df = pd.read_csv(df_path, sep="\t")
    treatment = raw_df['treatment'].unique()[0]
    sex = raw_df['sex'].unique()[0]
    time_point = raw_df['time_point'].unique()[0]
    print(f"treatment: {treatment}, sex: {sex}, time: {time_point}")

    # remove entries without atlas structure
    nonzero_mask = raw_df['atlas_structure_number'] != 0
    filtered_df = raw_df[nonzero_mask]

    # merge records from the same structure into count per structure
    count_per_value = filtered_df.groupby('atlas_structure_acronym').size().reset_index(name='count')
    count_per_value = count_per_value.sort_values(by='count', ascending=False)

    df = atlas.lookup_df
    all_levels_list = []

    for depth_level in range(10):  # depth of the atlas tree is 9
        descendants_at_depth = extract_descendants(tree, root_node_id, depth_level)
        descendants_at_depth = [df.loc[df['id'] == id, 'acronym'].values[0] for id in descendants_at_depth]
        ancestor_counts = {structure: {'cell_count': 0, 'density': 0, 'volume': 0} for structure in descendants_at_depth}

        for index, row in count_per_value.iterrows():
            atlas_structure_acronym = row['atlas_structure_acronym']
            count = row['count']
            structure_ancestors = atlas.get_structure_ancestors(atlas_structure_acronym)
            structure_ancestors.append(atlas_structure_acronym)
            for major_structure in descendants_at_depth:
                if major_structure in structure_ancestors:
                    ancestor_counts[major_structure]['cell_count'] += count

        voxel_volume = atlas.resolution[0] / 1000 * atlas.resolution[1] / 1000 * atlas.resolution[2] / 1000  # cubic mm
        for structure in ancestor_counts.keys():
            if structure in region_volumes:
                volume = region_volumes[structure]
            else:
                mask = atlas.get_structure_mask(structure)
                voxels = mask[mask > 0].shape[0]
                volume = voxels * voxel_volume
                region_volumes[structure] = volume
            ancestor_counts[structure]['volume'] = volume
            ancestor_counts[structure]['density'] = ancestor_counts[structure]['cell_count'] / volume

        ancestor_data_list = [{'Structure': ">".join([*atlas.get_structure_ancestors(ancestor), ancestor]), **counts} for ancestor, counts in ancestor_counts.items()]
        result_df = pd.DataFrame(ancestor_data_list)
        all_levels_list.append(result_df)

    all_levels_df = pd.concat(all_levels_list, ignore_index=True)
    all_levels_df.sort_values(by='Structure', inplace=True)
    all_levels_df.to_csv(df_path.replace(".csv", "all_structures_density.csv"))
    return treatment, sex, time_point, all_levels_df


def extract_descendants(tree, node_id, depth, current_depth=0):
    descendants = []

    if current_depth == depth:
        return [node_id]

    children = tree.children(node_id)
    for child in children:
        descendants.extend(extract_descendants(tree, child.identifier, depth, current_depth + 1))

    return descendants

In [9]:
# "csv_files" archive needs to be downloaded and unpacked (see README.md file for instructions)
brain_dfs_02CL89 = sorted(glob(os.path.join(os.getcwd(), "csv_files", "02CL89", "*with_metadata.csv")))
brain_dfs_03CL12 = sorted(glob(os.path.join(os.getcwd(), "csv_files", "03CL12", "*with_metadata.csv")))
brain_dfs_03CL47 = sorted(glob(os.path.join(os.getcwd(), "csv_files", "03CL47", "*with_metadata.csv")))

all_dfs = []
all_dfs.extend(brain_dfs_02CL89)
all_dfs.extend(brain_dfs_03CL12)
all_dfs.extend(brain_dfs_03CL47)

density_dfs = {}
for df_path in all_dfs:
    density_dfs[df_path] = {}
    treatment, sex, time_point, density_df = get_all_structures_cell_density(df_path)
    density_dfs[df_path]['treatment'] = treatment
    density_dfs[df_path]['sex'] = sex
    density_dfs[df_path]['time_point'] = time_point
    density_dfs[df_path]['df'] = density_df

# form "all morphine" and "all saline" DFs
    
morphine_dfs = []
saline_dfs = []
for df_path in all_dfs:
    df_path_dict = density_dfs[df_path]
    density_df = df_path_dict['df']
    treatment = df_path_dict['treatment']
    if treatment == "morphine":
        morphine_dfs.append(density_df)
    elif treatment == "saline":
        saline_dfs.append(density_df)

# create merged dfs (with repeating structures) - 1 for morphine, 1 for saline
morphine_dfs = pd.concat(morphine_dfs)
assert morphine_dfs.shape[0] == 840 * 15

saline_dfs = pd.concat(saline_dfs)
assert saline_dfs.shape[0] == 840 * 12

morphine_dfs.sort_values(by='Structure', inplace=True)
saline_dfs.sort_values(by='Structure', inplace=True)

morphine_dfs.reset_index(drop=True, inplace=True)
saline_dfs.reset_index(drop=True, inplace=True)

morphine_dfs['acronym'] = list(map(get_acronym, morphine_dfs['Structure'].to_list()))
saline_dfs['acronym'] = list(map(get_acronym, saline_dfs['Structure'].to_list()))

morphine_average_density_df = morphine_dfs.groupby('Structure')['density'].mean().reset_index()
saline_average_density_df = saline_dfs.groupby('Structure')['density'].mean().reset_index()
morphine_average_density_df['acronym'] = list(map(get_acronym, morphine_average_density_df['Structure'].to_list()))
saline_average_density_df['acronym'] = list(map(get_acronym, saline_average_density_df['Structure'].to_list()))

morphine_average_cell_count_df = morphine_dfs.groupby('Structure')['cell_count'].mean().reset_index()
saline_average_cell_count_df = saline_dfs.groupby('Structure')['cell_count'].mean().reset_index()
morphine_average_cell_count_df['acronym'] = list(map(get_acronym, morphine_average_cell_count_df['Structure'].to_list()))
saline_average_cell_count_df['acronym'] = list(map(get_acronym, saline_average_cell_count_df['Structure'].to_list()))

merged_df = pd.merge(morphine_average_density_df, saline_average_density_df, on='Structure', suffixes=('_morphine', '_saline'))

merged_df = merged_df[merged_df['density_saline'] > 0]

merged_df['density_ratio'] = merged_df['density_morphine'] / merged_df['density_saline']

density_ratio_df = merged_df[['Structure', 'density_ratio']]

# calculate p-values for every structure
p_values = []
significance = []
for structure in density_ratio_df['Structure'].to_list():
    morphine_dataset = morphine_dfs[morphine_dfs['Structure'] == structure]['density'].to_list()
    assert len(morphine_dataset) == 15
    saline_dataset = saline_dfs[saline_dfs['Structure'] == structure]['density'].to_list()
    assert len(saline_dataset) == 12
    p_val = get_p_value(morphine_dataset, saline_dataset)
    signif = p_val <= significance_threshold
    p_values.append(p_val)
    significance.append(signif)

density_ratio_df['p_value'] = p_values
density_ratio_df['significant'] = significance

density_ratio_df['acronym'] = list(map(get_acronym, density_ratio_df['Structure'].to_list()))

reading DF: /h20/home/iana/bio/notebooks/cebra/csv_files/02CL89/job_00833_df_for_dashboard_with_metadata.csv
treatment: morphine, sex: m, time: 1.0
reading DF: /h20/home/iana/bio/notebooks/cebra/csv_files/02CL89/job_00834_df_for_dashboard_with_metadata.csv
treatment: morphine, sex: m, time: 4.0
reading DF: /h20/home/iana/bio/notebooks/cebra/csv_files/02CL89/job_00835_df_for_dashboard_with_metadata.csv
treatment: saline, sex: m, time: 1.0
reading DF: /h20/home/iana/bio/notebooks/cebra/csv_files/02CL89/job_00836_df_for_dashboard_with_metadata.csv
treatment: saline, sex: m, time: 4.0
reading DF: /h20/home/iana/bio/notebooks/cebra/csv_files/03CL12/job_01093_df_for_dashboard_with_metadata.csv
treatment: morphine, sex: f, time: 4.0
reading DF: /h20/home/iana/bio/notebooks/cebra/csv_files/03CL12/job_01103_df_for_dashboard_with_metadata.csv
treatment: saline, sex: f, time: 4.0
reading DF: /h20/home/iana/bio/notebooks/cebra/csv_files/03CL12/job_01106_df_for_dashboard_with_metadata.csv
treatment

# 1. Comparing all morphine and all saline

### Number of structures higher in morphine or in saline

In [10]:
merged_df = pd.merge(morphine_average_density_df, saline_average_density_df, on='Structure', suffixes=('_morphine', '_saline'))
morphine_grater = merged_df[merged_df['density_morphine'] > merged_df['density_saline']]
print("Structures with higher morphine density", morphine_grater.shape[0])

Structures with higher morphine density 713


In [11]:
saline_grater = merged_df[merged_df['density_saline'] > merged_df['density_morphine']]
print("Structures with higher saline density", saline_grater.shape[0])

Structures with higher saline density 124


In [12]:
saline_grater['region'] = list(map(get_major_parent, saline_grater['acronym_saline'].to_list()))

count = Counter(saline_grater['region'])
count

Counter({'Other': 18,
         'MY': 2,
         'HY': 2,
         'TH': 17,
         'MB': 12,
         'CB': 3,
         'PAL': 5,
         'STR': 6,
         'HPF': 4,
         'Isocortex': 52,
         'OLF': 3})

In [13]:
print(count['TH'] / len(atlas.get_structure_descendants('TH')) * 100, "% of structures with decreased activation in Thalamus")

25.757575757575758 % of structures with decreased activation in Thalamus


### Table 1 - Large structures <a id="table1"/>

In [14]:
table_1 = density_ratio_df[density_ratio_df['acronym'].isin(large_structures)]
table_1.sort_values('density_ratio', inplace=True, ascending=False)

display(HTML(table_1.to_html()))

,Structure,density_ratio,p_value,significant,acronym
178,root>grey>BS>HB>P,3.762583,0.029661,True,P
120,root>grey>BS>HB>MY,2.802589,0.152434,False,MY
826,root>grey>CH>CTX>CTXsp,2.184512,0.030112,True,CTXsp
213,root>grey>BS>IB>HY,2.026222,0.106907,False,HY
475,root>grey>CH>CTX>CTXpl>HPF,1.824296,0.037559,True,HPF
450,root>grey>CH>CNU>STR,1.678736,0.226032,False,STR
435,root>grey>CH>CNU>PAL,1.633305,0.292115,False,PAL
508,root>grey>CH>CTX>CTXpl>Isocortex,1.627575,0.287906,False,Isocortex
337,root>grey>BS>MB,1.513873,0.186865,False,MB
803,root>grey>CH>CTX>CTXpl>OLF,1.481895,0.212637,False,OLF


### 1. Number of significant structures between 'all morphine' and 'all saline'

In [15]:
significant_structures = density_ratio_df[density_ratio_df['significant'] == True]['acronym'].to_list()

In [16]:
len(significant_structures)

120

In [17]:
filtered_significant_structures = [
    x for x in significant_structures if 
    is_large_enough(x)
    and is_grey(x)
    and both_above_threshold(x, morphine_average_cell_count_df, saline_average_cell_count_df)
]

In [18]:
len(filtered_significant_structures)

65

In [19]:
filtered_significant_structures_no_parents = exclude_parents(filtered_significant_structures)
len(filtered_significant_structures_no_parents)

46

In [20]:
sorted(filtered_significant_structures_no_parents)

['ACAd5',
 'ACAd6a',
 'ACAv2/3',
 'ACAv5',
 'ACAv6a',
 'AIp5',
 'AUDd5',
 'AUDp5',
 'AUDv5',
 'BLAp',
 'CA1',
 'CA2',
 'CA3',
 'CENT2',
 'CENT3',
 'DG-mo',
 'ECT5',
 'ECT6a',
 'ENTl5',
 'ENTl6a',
 'ENTm3',
 'ENTm5',
 'ENTm6',
 'EPd',
 'EPv',
 'HATA',
 'ICc',
 'ILA5',
 'MRN',
 'P-sat',
 'PB',
 'PL5',
 'PL6a',
 'PPN',
 'ProS',
 'SSp-bfd5',
 'SSs5',
 'SSs6a',
 'TEa5',
 'TEa6a',
 'VISC5',
 'VISC6a',
 'VISl5',
 'VISp5',
 'VISpor5',
 'VTA']

In [21]:
# Grouped by large brain regions

major_structures = [get_major_parent(x) for x in filtered_significant_structures_no_parents]

counts = Counter(major_structures)
print(counts)

Counter({'Isocortex': 24, 'HPF': 11, 'MB': 4, 'CTXsp': 3, 'P': 2, 'CB': 2})


### Table S1 - Structures with significant differences <a id="tables1"/>

In [85]:
table_s1 = density_ratio_df[density_ratio_df['acronym'].isin(filtered_significant_structures_no_parents)]
table_s1['region'] = list(map(get_major_parent, table_s1['acronym'].to_list()))
table_s1.sort_values('density_ratio', inplace=True, ascending=False, ignore_index=True)
table_s1['density_ratio'] = table_s1['density_ratio'].round(2)
table_s1['p_value'] = table_s1['p_value'].round(4)
display(HTML(table_s1[['acronym', 'region', 'density_ratio', 'p_value']].to_html()))

,acronym,region,density_ratio,p_value
0,AUDd5,Isocortex,6.05,0.0002
1,ACAv6a,Isocortex,5.87,0.0133
2,SSs5,Isocortex,5.68,0.0008
3,ACAv5,Isocortex,4.68,0.0112
4,VISC5,Isocortex,4.62,0.0039
5,PPN,MB,4.60,0.0330
6,PB,P,4.53,0.0025
7,P-sat,P,4.40,0.0352
8,EPd,CTXsp,4.21,0.0018
9,PL6a,Isocortex,4.07,0.0034


#### 1a. More cells on average in morphine brains

In [23]:
merged_df[merged_df['Structure'] == 'root']

,Structure,density_morphine,acronym_morphine,density_saline,acronym_saline
0,root,234.441718,root,139.981616,root


In [24]:
morphine_average_cell_count_df[morphine_average_cell_count_df['Structure'] == 'root']

,Structure,cell_count,acronym
0,root,118635.133333,root


In [25]:
saline_average_cell_count_df[saline_average_cell_count_df['Structure'] == 'root']

,Structure,cell_count,acronym
0,root,70835.25,root


### 2. Number of structures that tend to significance

In [26]:
close_to_significant_structures = density_ratio_df[(density_ratio_df['p_value'] >= 0.05) & (density_ratio_df['p_value'] < 0.1)]['acronym'].to_list()

In [27]:
filtered_close_to_significant_structures = [
    x for x in close_to_significant_structures if 
    is_large_enough(x)
    and is_grey(x)
    and both_above_threshold(x, morphine_average_cell_count_df, saline_average_cell_count_df)
]

In [28]:
len(filtered_close_to_significant_structures)

40

In [29]:
filtered_close_to_significant_structures_no_parents = exclude_parents(filtered_close_to_significant_structures)

In [30]:
filtered_close_to_significant_structures_no_parents.remove('ECT')  # has children in significant structures
filtered_close_to_significant_structures_no_parents.remove('ENT')  # has children in significant structures

In [31]:
len(filtered_close_to_significant_structures_no_parents)

31

In [32]:
sorted(filtered_close_to_significant_structures_no_parents)

['AAA',
 'ACAv1',
 'ACB',
 'AHN',
 'AId6a',
 'AIp6a',
 'AIv5',
 'AUDp',
 'BLAv',
 'CLA',
 'GU5',
 'LHA',
 'LPO',
 'MOp5',
 'MOs6a',
 'MPN',
 'MPO',
 'MV',
 'MY-sen',
 'ORBl6a',
 'P-mot',
 'PAG',
 'PL2/3',
 'RAmb',
 'RSPagl6a',
 'SCiw',
 'SI',
 'SSp-bfd6a',
 'SSp-m5',
 'VISli',
 'VISp6a']

### Table S2 - Structures that tend to significance <a id="tables2"/>

In [86]:
table_s2 = density_ratio_df[density_ratio_df['acronym'].isin(filtered_close_to_significant_structures_no_parents)]
table_s2['region'] = list(map(get_major_parent, table_s2['acronym'].to_list()))
table_s2.sort_values('density_ratio', inplace=True, ascending=False, ignore_index=True)
table_s2['density_ratio'] = table_s2['density_ratio'].round(2)
table_s2['p_value'] = table_s2['p_value'].round(3)
display(HTML(table_s2[['acronym', 'region', 'density_ratio', 'p_value']].to_html()))

,acronym,region,density_ratio,p_value
0,CLA,CTXsp,9.32,0.071
1,AIp6a,Isocortex,5.82,0.066
2,AId6a,Isocortex,4.90,0.051
3,GU5,Isocortex,4.24,0.057
4,RSPagl6a,Isocortex,4.14,0.066
5,MV,MY,3.99,0.097
6,MOp5,Isocortex,3.93,0.087
7,MOs6a,Isocortex,3.72,0.089
8,LPO,HY,3.70,0.053
9,MY-sen,MY,3.51,0.068


# 2. Comparing 1h and 4h

### 2a. There are more cells in morphine than in saline at 1h

In [34]:
# 1h

morphine_dfs_1h = []
saline_dfs_1h = []
for df_path in all_dfs:
    df_path_dict = density_dfs[df_path]
    density_df = df_path_dict['df']
    treatment = df_path_dict['treatment']
    time_point = df_path_dict['time_point']
    if treatment == "morphine" and time_point == 1:
        morphine_dfs_1h.append(density_df)
    elif treatment == "saline" and time_point == 1:
        saline_dfs_1h.append(density_df)
    else:
        pass

morphine_dfs_1h = pd.concat(morphine_dfs_1h)
assert morphine_dfs_1h.shape[0] == 840 * 8

saline_dfs_1h = pd.concat(saline_dfs_1h)
assert saline_dfs_1h.shape[0] == 840 * 5

morphine_dfs_1h.sort_values(by='Structure', inplace=True)
saline_dfs_1h.sort_values(by='Structure', inplace=True)

morphine_dfs_1h.reset_index(drop=True, inplace=True)
saline_dfs_1h.reset_index(drop=True, inplace=True)

morphine_1h_average_density_df = morphine_dfs_1h.groupby('Structure')['density'].mean().reset_index()
saline_1h_average_density_df = saline_dfs_1h.groupby('Structure')['density'].mean().reset_index()

morphine_1h_average_density_df['acronym'] = list(map(get_acronym, morphine_1h_average_density_df['Structure'].to_list()))
saline_1h_average_density_df['acronym'] = list(map(get_acronym, saline_1h_average_density_df['Structure'].to_list()))

morphine_average_cell_count_df_1h = morphine_dfs_1h.groupby('Structure')['cell_count'].mean().reset_index()
saline_average_cell_count_df_1h = saline_dfs_1h.groupby('Structure')['cell_count'].mean().reset_index()
morphine_average_cell_count_df_1h['acronym'] = list(map(get_acronym, morphine_average_cell_count_df_1h['Structure'].to_list()))
saline_average_cell_count_df_1h['acronym'] = list(map(get_acronym, saline_average_cell_count_df_1h['Structure'].to_list()))

merged_df_1h = pd.merge(morphine_1h_average_density_df, saline_1h_average_density_df, on='Structure', suffixes=('_morphine', '_saline'))

merged_df_1h = merged_df_1h[merged_df_1h['density_saline'] > 0]   # TODO: how do we deal with binary effects?

merged_df_1h['density_ratio'] = merged_df_1h['density_morphine'] / merged_df_1h['density_saline']

density_ratio_df_1h = merged_df_1h[['Structure', 'density_ratio']]

p_values = []
significance = []
for structure in density_ratio_df_1h['Structure'].to_list():
    morphine_dataset = morphine_dfs_1h[morphine_dfs_1h['Structure'] == structure]['density'].to_list()
    assert len(morphine_dataset) == 8
    saline_dataset = saline_dfs_1h[saline_dfs_1h['Structure'] == structure]['density'].to_list()
    assert len(saline_dataset) == 5
    p_val = get_p_value(morphine_dataset, saline_dataset)
    signif = p_val <= significance_threshold
    p_values.append(p_val)
    significance.append(signif)

density_ratio_df_1h['p_value'] = p_values
density_ratio_df_1h['significant'] = significance

density_ratio_df_1h['acronym'] = list(map(get_acronym, density_ratio_df_1h['Structure'].to_list()))


In [35]:
merged_df_1h[merged_df_1h['Structure'] == 'root']

,Structure,density_morphine,acronym_morphine,density_saline,acronym_saline,density_ratio
0,root,254.12843,root,206.29939,root,1.231843


In [36]:
morphine_average_cell_count_df_1h[morphine_average_cell_count_df_1h['Structure'] == 'root']

,Structure,cell_count,acronym
0,root,128597.25,root


In [37]:
saline_average_cell_count_df_1h[saline_average_cell_count_df_1h['Structure'] == 'root']

,Structure,cell_count,acronym
0,root,104394.2,root


### 2b. There are more cells in morphine than in saline at 4h

In [38]:
# 4h

morphine_dfs_4h = []
saline_dfs_4h = []
for df_path in all_dfs:
    df_path_dict = density_dfs[df_path]
    density_df = df_path_dict['df']
    treatment = df_path_dict['treatment']
    time_point = df_path_dict['time_point']
    if treatment == "morphine" and time_point == 4:
        morphine_dfs_4h.append(density_df)
    elif treatment == "saline" and time_point == 4:
        saline_dfs_4h.append(density_df)
    else:
        pass

morphine_dfs_4h = pd.concat(morphine_dfs_4h)
assert morphine_dfs_4h.shape[0] == 840 * 7

saline_dfs_4h = pd.concat(saline_dfs_4h)
assert saline_dfs_4h.shape[0] == 840 * 7

morphine_dfs_4h.sort_values(by='Structure', inplace=True)
saline_dfs_4h.sort_values(by='Structure', inplace=True)

morphine_dfs_4h.reset_index(drop=True, inplace=True)
saline_dfs_4h.reset_index(drop=True, inplace=True)

morphine_4h_average_density_df = morphine_dfs_4h.groupby('Structure')['density'].mean().reset_index()
saline_4h_average_density_df = saline_dfs_4h.groupby('Structure')['density'].mean().reset_index()

morphine_4h_average_density_df['acronym'] = list(map(get_acronym, morphine_4h_average_density_df['Structure'].to_list()))
saline_4h_average_density_df['acronym'] = list(map(get_acronym, saline_4h_average_density_df['Structure'].to_list()))

morphine_average_cell_count_df_4h = morphine_dfs_4h.groupby('Structure')['cell_count'].mean().reset_index()
saline_average_cell_count_df_4h = saline_dfs_4h.groupby('Structure')['cell_count'].mean().reset_index()
morphine_average_cell_count_df_4h['acronym'] = list(map(get_acronym, morphine_average_cell_count_df_4h['Structure'].to_list()))
saline_average_cell_count_df_4h['acronym'] = list(map(get_acronym, saline_average_cell_count_df_4h['Structure'].to_list()))

merged_df_4h = pd.merge(morphine_4h_average_density_df, saline_4h_average_density_df, on='Structure', suffixes=('_morphine', '_saline'))

merged_df_4h = merged_df_4h[merged_df_4h['density_saline'] > 0]   # TODO: how do we deal with binary effects?

merged_df_4h['density_ratio'] = merged_df_4h['density_morphine'] / merged_df_4h['density_saline']

density_ratio_df_4h = merged_df_4h[['Structure', 'density_ratio']]

p_values = []
significance = []
for structure in density_ratio_df_4h['Structure'].to_list():
    morphine_dataset = morphine_dfs_4h[morphine_dfs_4h['Structure'] == structure]['density'].to_list()
    assert len(morphine_dataset) == 7
    saline_dataset = saline_dfs_4h[saline_dfs_4h['Structure'] == structure]['density'].to_list()
    assert len(saline_dataset) == 7
    p_val = get_p_value(morphine_dataset, saline_dataset)
    signif = p_val <= significance_threshold
    p_values.append(p_val)
    significance.append(signif)

density_ratio_df_4h['p_value'] = p_values
density_ratio_df_4h['significant'] = significance

density_ratio_df_4h['acronym'] = list(map(get_acronym, density_ratio_df_4h['Structure'].to_list()))

In [39]:
merged_df_4h[merged_df_4h['Structure'] == 'root']

,Structure,density_morphine,acronym_morphine,density_saline,acronym_saline,density_ratio
0,root,211.942618,root,92.611778,root,2.288506


In [40]:
morphine_average_cell_count_df_4h[morphine_average_cell_count_df_4h['Structure'] == 'root']

,Structure,cell_count,acronym
0,root,107249.857143,root


In [41]:
saline_average_cell_count_df_4h[saline_average_cell_count_df_4h['Structure'] == 'root']

,Structure,cell_count,acronym
0,root,46864.571429,root


### 2c. Number of structures significant at 1h (between Morphine and Saline)

In [42]:
significant_structures_1h = density_ratio_df_1h[density_ratio_df_1h['significant'] == True]['acronym'].to_list()

filtered_significant_structures_1h = [
    x for x in significant_structures_1h if 
    is_large_enough(x)
    and is_grey(x)
    and both_above_threshold(x, morphine_average_cell_count_df_1h, saline_average_cell_count_df_1h)
]

filtered_significant_structures_no_parents_1h = exclude_parents(filtered_significant_structures_1h)
print("Structures with significant difference:", len(filtered_significant_structures_no_parents_1h))

Structures with significant difference: 12


In [43]:
print(sorted(filtered_significant_structures_no_parents_1h))

['AUDd5', 'CA3', 'ENTl5', 'ENTm3', 'ENTm5', 'ENTm6', 'EPd', 'MRN', 'PB', 'SSs5', 'VISp5', 'VISpor5']


### Table S4 <a id="tables4"/>

In [87]:
table_s4 = density_ratio_df_1h[density_ratio_df_1h['acronym'].isin(filtered_significant_structures_no_parents_1h)]
# table_s4['region'] = list(map(get_major_parent, table_s4['acronym'].to_list()))
table_s4.sort_values('density_ratio', inplace=True, ascending=False, ignore_index=True)
table_s4['density_ratio'] = table_s4['density_ratio'].round(2)
table_s4['p_value'] = table_s4['p_value'].round(4)
display(HTML(table_s4[['acronym', 'density_ratio', 'p_value']].to_html()))

,acronym,density_ratio,p_value
0,PB,11.11,0.0444
1,EPd,5.27,0.0235
2,ENTl5,4.37,0.0080
3,ENTm3,3.94,0.0409
4,ENTm5,3.81,0.0347
5,VISpor5,3.51,0.0413
6,AUDd5,3.48,0.0197
7,SSs5,3.42,0.0365
8,VISp5,3.40,0.0310
9,ENTm6,3.14,0.0374


### 4. Number of structures significant at 4h (between Morphine and Saline)

In [45]:
significant_structures_4h = density_ratio_df_4h[density_ratio_df_4h['significant'] == True]['acronym'].to_list()

filtered_significant_structures_4h = [
    x for x in significant_structures_4h if 
    is_large_enough(x)
    and is_grey(x)
    and both_above_threshold(x, morphine_average_cell_count_df_4h, saline_average_cell_count_df_4h)
]
print(len(filtered_significant_structures_4h))

filtered_significant_structures_no_parents_4h = exclude_parents(filtered_significant_structures_4h)
print("Structures with significant difference:", len(filtered_significant_structures_no_parents_4h))

47
Structures with significant difference: 34


In [46]:
sorted(filtered_significant_structures_no_parents_4h)

['ACAd',
 'ACAv5',
 'ACB',
 'AIp5',
 'AUDd',
 'AUDp5',
 'AUDpo',
 'AUDv5',
 'CENT2',
 'DMH',
 'ECT5',
 'ECT6a',
 'ENTl5',
 'ENTl6a',
 'ENTm5',
 'ENTm6',
 'EPd',
 'GU5',
 'ICc',
 'ILA5',
 'PB',
 'PL5',
 'PL6a',
 'SSs5',
 'SSs6a',
 'TEa5',
 'TEa6a',
 'VISC5',
 'VISC6a',
 'VISl5',
 'VISli',
 'VISp5',
 'VISp6a',
 'VISpor5']

### Table S5 <a id="tables5"/>

In [88]:
table_s5 = density_ratio_df_4h[density_ratio_df_4h['acronym'].isin(filtered_significant_structures_no_parents_4h)]
# table_s5['region'] = list(map(get_major_parent, table_s5['acronym'].to_list()))
table_s5.sort_values('density_ratio', inplace=True, ascending=False, ignore_index=True)
table_s5['density_ratio'] = table_s5['density_ratio'].round(2)
table_s5['p_value'] = table_s5['p_value'].round(4)
display(HTML(table_s5[['acronym', 'density_ratio', 'p_value']].to_html()))

,acronym,density_ratio,p_value
0,PL6a,15.16,0.0052
1,ACAv5,13.52,0.0328
2,SSs5,10.87,0.0185
3,VISl5,7.99,0.0369
4,VISC5,7.85,0.0033
5,SSs6a,7.41,0.0373
6,AUDp5,7.29,0.0002
7,GU5,6.24,0.0390
8,AUDv5,5.97,0.0004
9,DMH,5.87,0.0402


# 3. Comparing male and female mice

### Number of structures significant for male mice (between morphine and saline)

In [48]:
# male

morphine_dfs_male = []
saline_dfs_male = []
for df_path in all_dfs:
    df_path_dict = density_dfs[df_path]
    density_df = df_path_dict['df']
    treatment = df_path_dict['treatment']
    sex = df_path_dict['sex']
    if treatment == "morphine" and sex == 'm':
        morphine_dfs_male.append(density_df)
    elif treatment == "saline" and sex == 'm':
        saline_dfs_male.append(density_df)
    else:
        pass

morphine_dfs_male = pd.concat(morphine_dfs_male)
assert morphine_dfs_male.shape[0] == 840 * 9

saline_dfs_male = pd.concat(saline_dfs_male)
assert saline_dfs_male.shape[0] == 840 * 6

morphine_dfs_male.sort_values(by='Structure', inplace=True)
saline_dfs_male.sort_values(by='Structure', inplace=True)

morphine_dfs_male.reset_index(drop=True, inplace=True)
saline_dfs_male.reset_index(drop=True, inplace=True)

morphine_male_average_density_df = morphine_dfs_male.groupby('Structure')['density'].mean().reset_index()
saline_male_average_density_df = saline_dfs_male.groupby('Structure')['density'].mean().reset_index()

morphine_male_average_density_df['acronym'] = list(map(get_acronym, morphine_male_average_density_df['Structure'].to_list()))
saline_male_average_density_df['acronym'] = list(map(get_acronym, saline_male_average_density_df['Structure'].to_list()))

morphine_average_cell_count_df_male = morphine_dfs_male.groupby('Structure')['cell_count'].mean().reset_index()
saline_average_cell_count_df_male = saline_dfs_male.groupby('Structure')['cell_count'].mean().reset_index()
morphine_average_cell_count_df_male['acronym'] = list(map(get_acronym, morphine_average_cell_count_df_male['Structure'].to_list()))
saline_average_cell_count_df_male['acronym'] = list(map(get_acronym, saline_average_cell_count_df_male['Structure'].to_list()))

merged_df_male = pd.merge(morphine_male_average_density_df, saline_male_average_density_df, on='Structure', suffixes=('_morphine', '_saline'))

merged_df_male = merged_df_male[merged_df_male['density_saline'] > 0]   # TODO: how do we deal with binary effects?
merged_df_male = merged_df_male[merged_df_male['density_morphine'] > 0]   # TODO: how do we deal with binary effects?

merged_df_male['density_ratio'] = merged_df_male['density_morphine'] / merged_df_male['density_saline']

density_ratio_df_male = merged_df_male[['Structure', 'density_ratio']]

p_values = []
significance = []
for structure in density_ratio_df_male['Structure'].to_list():
    morphine_dataset_m = morphine_dfs_male[morphine_dfs_male['Structure'] == structure]['density'].to_list()
    assert len(morphine_dataset_m) == 9
    saline_dataset_m = saline_dfs_male[saline_dfs_male['Structure'] == structure]['density'].to_list()
    assert len(saline_dataset_m) == 6
    p_val = get_p_value(morphine_dataset_m, saline_dataset_m)
    signif = p_val <= significance_threshold
    p_values.append(p_val)
    significance.append(signif)

density_ratio_df_male['p_value'] = p_values
density_ratio_df_male['significant'] = significance
density_ratio_df_male['acronym'] = list(map(get_acronym, density_ratio_df_male['Structure'].to_list()))

significant_structures_male = density_ratio_df_male[density_ratio_df_male['significant'] == True]['acronym'].to_list()

filtered_significant_structures_male = [
    x for x in significant_structures_male if 
    is_large_enough(x)
    and is_grey(x)
    and both_above_threshold(x, morphine_average_cell_count_df_male, saline_average_cell_count_df_male)
]
print(len(filtered_significant_structures_male))

filtered_significant_structures_no_parents_male = exclude_parents(filtered_significant_structures_male)
print("Structures with significant difference:", len(filtered_significant_structures_no_parents_male))

45
Structures with significant difference: 32


In [49]:
sorted(filtered_significant_structures_no_parents_male)

['ACAd6a',
 'ACAv5',
 'ACB',
 'AIp5',
 'AUDd5',
 'AUDp5',
 'AUDv5',
 'BLAp',
 'BMAp',
 'CA1',
 'CA3',
 'CENT2',
 'ECT6a',
 'ENTl5',
 'ENTl6a',
 'ENTm5',
 'ENTm6',
 'EPd',
 'LA',
 'LHA',
 'PA',
 'PAL',
 'PB',
 'PL5',
 'PL6a',
 'ProS',
 'SSs5',
 'SUB',
 'TEa6a',
 'VISC5',
 'VISl5',
 'VISpor5']

### Table S6 <a id="tables6"/>

In [89]:
table_s6 = density_ratio_df_male[density_ratio_df_male['acronym'].isin(filtered_significant_structures_no_parents_male)]
table_s6.sort_values('density_ratio', inplace=True, ascending=False, ignore_index=True)
table_s6['density_ratio'] = table_s6['density_ratio'].round(2)
table_s6['p_value'] = table_s6['p_value'].round(4)
display(HTML(table_s6[['acronym', 'density_ratio', 'p_value']].to_html()))

,acronym,density_ratio,p_value
0,ACB,15.60,0.0098
1,ACAv5,10.29,0.0486
2,LA,9.47,0.0247
3,PAL,8.49,0.0350
4,EPd,7.83,0.0089
5,BLAp,7.72,0.0176
6,AUDd5,7.00,0.0064
7,PL6a,6.91,0.0295
8,ACAd6a,6.86,0.0450
9,VISl5,5.95,0.0273


### Number of structures significant for female mice (between morphine and saline)

In [51]:
# female

morphine_dfs_female = []
saline_dfs_female = []
for df_path in all_dfs:
    df_path_dict = density_dfs[df_path]
    density_df = df_path_dict['df']
    treatment = df_path_dict['treatment']
    sex = df_path_dict['sex']
    if treatment == "morphine" and sex == 'f':
        morphine_dfs_female.append(density_df)
    elif treatment == "saline" and sex == 'f':
        saline_dfs_female.append(density_df)
    else:
        pass

morphine_dfs_female = pd.concat(morphine_dfs_female)
assert morphine_dfs_female.shape[0] == 840 * 6

saline_dfs_female = pd.concat(saline_dfs_female)
assert saline_dfs_female.shape[0] == 840 * 6

morphine_dfs_female.sort_values(by='Structure', inplace=True)
saline_dfs_female.sort_values(by='Structure', inplace=True)

morphine_dfs_female.reset_index(drop=True, inplace=True)
saline_dfs_female.reset_index(drop=True, inplace=True)

morphine_female_average_density_df = morphine_dfs_female.groupby('Structure')['density'].mean().reset_index()
saline_female_average_density_df = saline_dfs_female.groupby('Structure')['density'].mean().reset_index()

morphine_female_average_density_df['acronym'] = list(map(get_acronym, morphine_female_average_density_df['Structure'].to_list()))
saline_female_average_density_df['acronym'] = list(map(get_acronym, saline_female_average_density_df['Structure'].to_list()))

morphine_average_cell_count_df_female = morphine_dfs_female.groupby('Structure')['cell_count'].mean().reset_index()
saline_average_cell_count_df_female = saline_dfs_female.groupby('Structure')['cell_count'].mean().reset_index()
morphine_average_cell_count_df_female['acronym'] = list(map(get_acronym, morphine_average_cell_count_df_female['Structure'].to_list()))
saline_average_cell_count_df_female['acronym'] = list(map(get_acronym, saline_average_cell_count_df_female['Structure'].to_list()))

merged_df_female = pd.merge(morphine_female_average_density_df, saline_female_average_density_df, on='Structure', suffixes=('_morphine', '_saline'))

merged_df_female = merged_df_female[merged_df_female['density_morphine'] > 0]   # TODO: how do we deal with binary effects?
merged_df_female = merged_df_female[merged_df_female['density_saline'] > 0]   # TODO: how do we deal with binary effects?

merged_df_female['density_ratio'] = merged_df_female['density_morphine'] / merged_df_female['density_saline']

density_ratio_df_female = merged_df_female[['Structure', 'density_ratio']]

p_values = []
significance = []
for structure in density_ratio_df_female['Structure'].to_list():
    morphine_dataset_f = morphine_dfs_female[morphine_dfs_female['Structure'] == structure]['density'].to_list()
    assert len(morphine_dataset_f) == 6
    saline_dataset_f = saline_dfs_female[saline_dfs_female['Structure'] == structure]['density'].to_list()
    assert len(saline_dataset_f) == 6
    p_val = get_p_value(morphine_dataset_f, saline_dataset_f)
    signif = p_val <= significance_threshold
    p_values.append(p_val)
    significance.append(signif)

density_ratio_df_female['p_value'] = p_values
density_ratio_df_female['significant'] = significance
density_ratio_df_female['acronym'] = list(map(get_acronym, density_ratio_df_female['Structure'].to_list()))

significant_structures_female = density_ratio_df_female[density_ratio_df_female['significant'] == True]['acronym'].to_list()

filtered_significant_structures_female = [
    x for x in significant_structures_female if 
    is_large_enough(x)
    and is_grey(x)
    and both_above_threshold(x, morphine_average_cell_count_df_female, saline_average_cell_count_df_female)
]

filtered_significant_structures_no_parents_female = exclude_parents(filtered_significant_structures_female)
print("Structures with significant difference:", len(filtered_significant_structures_no_parents_female))

Structures with significant difference: 31


In [52]:
sorted(filtered_significant_structures_no_parents_female)

['AIv5',
 'AUDd5',
 'AUDp2/3',
 'AUDp5',
 'AUDpo',
 'AUDv5',
 'CA2',
 'CA3',
 'ECT2/3',
 'ECT5',
 'ENTl3',
 'ENTl5',
 'ENTm1',
 'ENTm3',
 'ICc',
 'ICe',
 'PB',
 'PERI2/3',
 'RSPagl5',
 'RSPd5',
 'SSp-bfd5',
 'SSp-tr',
 'SSs5',
 'TEa2/3',
 'TEa4',
 'TEa5',
 'VISC5',
 'VISa',
 'VISp5',
 'VISpor5',
 'VISrl']

### Table S7 <a id="tables7"/>

In [90]:
table_s7 = density_ratio_df_female[density_ratio_df_female['acronym'].isin(filtered_significant_structures_no_parents_female)]
table_s7.sort_values('density_ratio', inplace=True, ascending=False, ignore_index=True)
table_s7['density_ratio'] = table_s7['density_ratio'].round(2)
table_s7['p_value'] = table_s7['p_value'].round(4)
display(HTML(table_s7[['acronym', 'density_ratio', 'p_value']].to_html()))

,acronym,density_ratio,p_value
0,TEa4,6.87,0.0003
1,AUDp2/3,6.53,0.0027
2,ENTm1,5.36,0.0301
3,RSPagl5,4.84,0.0069
4,SSs5,4.67,0.0232
5,VISa,4.62,0.0038
6,AUDd5,4.57,0.0075
7,AUDv5,4.41,0.0010
8,VISC5,4.39,0.0206
9,TEa2/3,4.32,0.0125


#### Significant structures shared between males and females

In [54]:
set(filtered_significant_structures_no_parents_female).intersection(set(filtered_significant_structures_no_parents_male))

{'AUDd5', 'AUDp5', 'AUDv5', 'CA3', 'ENTl5', 'PB', 'SSs5', 'VISC5', 'VISpor5'}

# Binary effects

### Table S3 - binary effects <a id="tables3"/>

#### All morphine vs all saline

In [95]:
# split dfs into morphine and saline
morphine_dfs = []
saline_dfs = []
for df_path in all_dfs:
    df_path_dict = density_dfs[df_path]
    density_df = df_path_dict['df']
    treatment = df_path_dict['treatment']
    if treatment == "morphine":
        morphine_dfs.append(density_df)
    elif treatment == "saline":
        saline_dfs.append(density_df)

# create merged dfs (with repeating structures) - 1 for morphine, 1 for saline
morphine_dfs = pd.concat(morphine_dfs)
assert morphine_dfs.shape[0] == 840 * 15

saline_dfs = pd.concat(saline_dfs)
assert saline_dfs.shape[0] == 840 * 12

morphine_dfs.sort_values(by='Structure', inplace=True)
saline_dfs.sort_values(by='Structure', inplace=True)

morphine_dfs.reset_index(drop=True, inplace=True)
saline_dfs.reset_index(drop=True, inplace=True)

morphine_average_cell_count_df = morphine_dfs.groupby('Structure')['cell_count'].mean().reset_index()
saline_average_cell_count_df = saline_dfs.groupby('Structure')['cell_count'].mean().reset_index()

merged_df = pd.merge(morphine_average_cell_count_df, saline_average_cell_count_df, on='Structure', suffixes=('_morphine', '_saline'))
merged_df = merged_df[['Structure', 'cell_count_morphine', 'cell_count_saline']]

get_acronym = lambda x: x.split(">")[-1]
merged_df['acronym'] = list(map(get_acronym, merged_df['Structure'].to_list()))

def is_grey(acronym):
    parents = atlas.get_structure_ancestors(acronym)
    return "grey" in parents

merged_df['is_grey'] = list(map(is_grey, merged_df['acronym'].to_list()))
merged_df = merged_df[merged_df['is_grey'] == True]

binary_1 = []  # morphine only (red)
binary_2 = []  # saline only (blue)
# binary_3 = []  # both are zero (grey)

for ind, row in merged_df.iterrows():
    if (row['cell_count_saline'] < 1) and (row['cell_count_morphine'] >= 1):
        binary_1.append(row['acronym'])
    elif (row['cell_count_morphine'] < 1) and (row['cell_count_saline'] >= 1):
        binary_2.append(row['acronym'])
#     elif (row['cell_count_saline'] < 1) and (row['cell_count_morphine'] < 1):
#         binary_3.append(row['acronym'])
# print("Morphine only", binary_1)
# print("Saline only", binary_2)
# print("No cells in either", binary_3)

binary1_parents = list(map(get_major_parent, binary_1))
binary2_parents = list(map(get_major_parent, binary_2))

grouped1 = defaultdict(list)
grouped2 = defaultdict(list)

for c, p in zip(binary_1, binary1_parents):
    grouped1[p].append(c)
row1 = ""
for p, children in grouped1.items():
    row1 = row1 + f"{p}: {', '.join(children)}\n"

for c, p in zip(binary_2, binary2_parents):
    grouped2[p].append(c)
row2 = ""
for p, children in grouped2.items():
    row2 = row2 + f"{p}: {', '.join(children)}\n"

table_data = {
    "Morphine only": [row1],
    "Saline only": [row2]
}
table_df = pd.DataFrame(table_data)

html_table = table_df.to_html(index=False, escape=False).replace("\\n", "<br>")
display(HTML(html_table))

Morphine only,Saline only
"MY: AMBd, AMBv, LIN, LRNp, PAS, VI, y, RO, AP, GR, ECU, Pa5P: PC5, SG, V, SLCHY: ME, SFO, ASOTH: RHMB: DT, III, LT, MA3, Su3, PN, IPA, IPDMIsocortex: ACAd6b, AId6b, GU6b, ORBm6b, ORBvl6b, PL6b, SSp-ll6b, SSp-n6b, SSp-tr6b, SSp-ul6b, SSp-un6b",MY: RPAMB: SCO


#### 1h morphine vs 1h saline

In [96]:
morphine_dfs_1h = []
saline_dfs_1h = []
for df_path in all_dfs:
    df_path_dict = density_dfs[df_path]
    density_df = df_path_dict['df']
    treatment = df_path_dict['treatment']
    time_point = df_path_dict['time_point']
    if treatment == "morphine" and time_point == 1:
        morphine_dfs_1h.append(density_df)
    elif treatment == "saline" and time_point == 1:
        saline_dfs_1h.append(density_df)

# create merged dfs (with repeating structures) - 1 for morphine, 1 for saline
morphine_dfs_1h = pd.concat(morphine_dfs_1h)
assert morphine_dfs_1h.shape[0] == 840 * 8

saline_dfs_1h = pd.concat(saline_dfs_1h)
assert saline_dfs_1h.shape[0] == 840 * 5

morphine_dfs_1h.sort_values(by='Structure', inplace=True)
saline_dfs_1h.sort_values(by='Structure', inplace=True)

morphine_dfs_1h.reset_index(drop=True, inplace=True)
saline_dfs_1h.reset_index(drop=True, inplace=True)

morphine_average_cell_count_df = morphine_dfs_1h.groupby('Structure')['cell_count'].mean().reset_index()
saline_average_cell_count_df = saline_dfs_1h.groupby('Structure')['cell_count'].mean().reset_index()

merged_df = pd.merge(morphine_average_cell_count_df, saline_average_cell_count_df, on='Structure', suffixes=('_morphine', '_saline'))
merged_df = merged_df[['Structure', 'cell_count_morphine', 'cell_count_saline']]

get_acronym = lambda x: x.split(">")[-1]
merged_df['acronym'] = list(map(get_acronym, merged_df['Structure'].to_list()))

def is_grey(acronym):
    parents = atlas.get_structure_ancestors(acronym)
    return "grey" in parents

merged_df['is_grey'] = list(map(is_grey, merged_df['acronym'].to_list()))
merged_df = merged_df[merged_df['is_grey'] == True]

binary_1 = []  # morphine only (red)
binary_2 = []  # saline only (blue)
binary_3 = []  # both are zero (grey)

for ind, row in merged_df.iterrows():
    if (row['cell_count_saline'] < 1) and (row['cell_count_morphine'] >= 1):
        binary_1.append(row['acronym'])
    elif (row['cell_count_morphine'] < 1) and (row['cell_count_saline'] >= 1):
        binary_2.append(row['acronym'])
    elif (row['cell_count_saline'] < 1) and (row['cell_count_morphine'] < 1):
        binary_3.append(row['acronym'])
# print("Morphine only", binary_1)
# print("Saline only", binary_2)
# print("No cells in either", binary_3)

binary1_parents = list(map(get_major_parent, binary_1))
binary2_parents = list(map(get_major_parent, binary_2))

grouped1 = defaultdict(list)
grouped2 = defaultdict(list)

for c, p in zip(binary_1, binary1_parents):
    grouped1[p].append(c)
row1 = ""
for p, children in grouped1.items():
    row1 = row1 + f"{p}: {', '.join(children)}\n"

for c, p in zip(binary_2, binary2_parents):
    grouped2[p].append(c)
row2 = ""
for p, children in grouped2.items():
    row2 = row2 + f"{p}: {', '.join(children)}\n"

table_data = {
    "Morphine only": [row1],
    "Saline only": [row2]
}
table_df = pd.DataFrame(table_data)

html_table = table_df.to_html(index=False, escape=False).replace("\\n", "<br>")
display(HTML(html_table))

Morphine only,Saline only
"MY: AMB, AMBd, AMBv, IO, LIN, LRN, LRNm, LRNp, PGRNl, PPY, XII, x, MY-sat, RM, VCO, GR, ECU, Pa5P: PC5, SUT, V, LC, NI, RPO, SLC, SLD, KFHY: MMd, ASOTH: IAM, IntG, PCN, PR, SMT, VPLpcMB: LT, MA3, MT, Su3, PN, MEV, IPDM, IPRLIsocortex: AIv6b, ORBl6b, PL6b, SSp-ll6b, SSp-tr6b, SSp-ul6b, SSp-un6b","MY: PASHY: OV, VMPOSTR: BA"


#### 4h morphine vs 4h saline

In [97]:
morphine_dfs_4h = []
saline_dfs_4h = []
for df_path in all_dfs:
    df_path_dict = density_dfs[df_path]
    density_df = df_path_dict['df']
    treatment = df_path_dict['treatment']
    time_point = df_path_dict['time_point']
    if treatment == "morphine" and time_point == 4:
        morphine_dfs_4h.append(density_df)
    elif treatment == "saline" and time_point == 4:
        saline_dfs_4h.append(density_df)

# create merged dfs (with repeating structures) - 1 for morphine, 1 for saline
morphine_dfs_4h = pd.concat(morphine_dfs_4h)
assert morphine_dfs_4h.shape[0] == 840 * 7

saline_dfs_4h = pd.concat(saline_dfs_4h)
assert saline_dfs_4h.shape[0] == 840 * 7

morphine_dfs_4h.sort_values(by='Structure', inplace=True)
saline_dfs_4h.sort_values(by='Structure', inplace=True)

morphine_dfs_4h.reset_index(drop=True, inplace=True)
saline_dfs_4h.reset_index(drop=True, inplace=True)

morphine_average_cell_count_df = morphine_dfs_4h.groupby('Structure')['cell_count'].mean().reset_index()
saline_average_cell_count_df = saline_dfs_4h.groupby('Structure')['cell_count'].mean().reset_index()

merged_df = pd.merge(morphine_average_cell_count_df, saline_average_cell_count_df, on='Structure', suffixes=('_morphine', '_saline'))
merged_df = merged_df[['Structure', 'cell_count_morphine', 'cell_count_saline']]

get_acronym = lambda x: x.split(">")[-1]
merged_df['acronym'] = list(map(get_acronym, merged_df['Structure'].to_list()))

def is_grey(acronym):
    parents = atlas.get_structure_ancestors(acronym)
    return "grey" in parents

merged_df['is_grey'] = list(map(is_grey, merged_df['acronym'].to_list()))
merged_df = merged_df[merged_df['is_grey'] == True]

binary_1 = []  # morphine only (red)
binary_2 = []  # saline only (blue)
binary_3 = []  # both are zero (grey)

for ind, row in merged_df.iterrows():
    if (row['cell_count_saline'] < 1) and (row['cell_count_morphine'] >= 1):
        binary_1.append(row['acronym'])
    elif (row['cell_count_morphine'] < 1) and (row['cell_count_saline'] >= 1):
        binary_2.append(row['acronym'])
    elif (row['cell_count_saline'] < 1) and (row['cell_count_morphine'] < 1):
        binary_3.append(row['acronym'])
# print("Morphine only", binary_1)
# print("Saline only", binary_2)
# print("No cells in either", binary_3)

binary1_parents = list(map(get_major_parent, binary_1))
binary2_parents = list(map(get_major_parent, binary_2))

grouped1 = defaultdict(list)
grouped2 = defaultdict(list)

for c, p in zip(binary_1, binary1_parents):
    grouped1[p].append(c)
row1 = ""
for p, children in grouped1.items():
    row1 = row1 + f"{p}: {', '.join(children)}\n"

for c, p in zip(binary_2, binary2_parents):
    grouped2[p].append(c)
row2 = ""
for p, children in grouped2.items():
    row2 = row2 + f"{p}: {', '.join(children)}\n"

table_data = {
    "Morphine only": [row1],
    "Saline only": [row2]
}
table_df = pd.DataFrame(table_data)

html_table = table_df.to_html(index=False, escape=False).replace("\\n", "<br>")
display(HTML(html_table))

Morphine only,Saline only
"MY: PAS, NR, VI, RO, AP, GR, ECUP: P5, PC5, SG, V, SLCHY: ME, TMd, OV, SCH, SFO, VLPO, VMPO, ARH, ASO, PVaMB: DT, III, LT, MA3, Su3, PN, CLI, IPA, IPDM, IPIPAL: BACSTR: BAIsocortex: ACAd6b, AId6b, AUDd6b, GU6b, ILA6b, MOp6b, MOs6b, ORBm6b, ORBvl6b, PL6b, VISa6b, VISrl6b, RSPd6b, RSPv6b, SSp-ll6b, SSp-m6b, SSp-n6b, SSp-tr6b, SSp-ul6b, SSp-un6b, VISpl6b","MY: LIN, RPATH: IGL, LGd-ip, LGd-shMB: SCOIsocortex: VISam6b, VISpm6b"


#### Male morphine vs male saline

In [98]:
morphine_dfs_male = []
saline_dfs_male = []
for df_path in all_dfs:
    df_path_dict = density_dfs[df_path]
    density_df = df_path_dict['df']
    treatment = df_path_dict['treatment']
    sex = df_path_dict['sex']
    if treatment == "morphine" and sex == 'm':
        morphine_dfs_male.append(density_df)
    elif treatment == "saline" and sex == 'm':
        saline_dfs_male.append(density_df)

# create merged dfs (with repeating structures) - 1 for morphine, 1 for saline
morphine_dfs_male = pd.concat(morphine_dfs_male)
assert morphine_dfs_male.shape[0] == 840 * 9

saline_dfs_male = pd.concat(saline_dfs_male)
assert saline_dfs_male.shape[0] == 840 * 6

morphine_dfs_male.sort_values(by='Structure', inplace=True)
saline_dfs_male.sort_values(by='Structure', inplace=True)

morphine_dfs_male.reset_index(drop=True, inplace=True)
saline_dfs_male.reset_index(drop=True, inplace=True)

morphine_average_cell_count_df = morphine_dfs_male.groupby('Structure')['cell_count'].mean().reset_index()
saline_average_cell_count_df = saline_dfs_male.groupby('Structure')['cell_count'].mean().reset_index()

merged_df = pd.merge(morphine_average_cell_count_df, saline_average_cell_count_df, on='Structure', suffixes=('_morphine', '_saline'))
merged_df = merged_df[['Structure', 'cell_count_morphine', 'cell_count_saline']]

get_acronym = lambda x: x.split(">")[-1]
merged_df['acronym'] = list(map(get_acronym, merged_df['Structure'].to_list()))

def is_grey(acronym):
    parents = atlas.get_structure_ancestors(acronym)
    return "grey" in parents

merged_df['is_grey'] = list(map(is_grey, merged_df['acronym'].to_list()))
merged_df = merged_df[merged_df['is_grey'] == True]

binary_1 = []  # morphine only (red)
binary_2 = []  # saline only (blue)
binary_3 = []  # both are zero (grey)

for ind, row in merged_df.iterrows():
    if (row['cell_count_saline'] < 1) and (row['cell_count_morphine'] >= 1):
        binary_1.append(row['acronym'])
    elif (row['cell_count_morphine'] < 1) and (row['cell_count_saline'] >= 1):
        binary_2.append(row['acronym'])
    elif (row['cell_count_saline'] < 1) and (row['cell_count_morphine'] < 1):
        binary_3.append(row['acronym'])
# print("Morphine only", binary_1)
# print("Saline only", binary_2)
# print("No cells in either", binary_3)

binary1_parents = list(map(get_major_parent, binary_1))
binary2_parents = list(map(get_major_parent, binary_2))

grouped1 = defaultdict(list)
grouped2 = defaultdict(list)

for c, p in zip(binary_1, binary1_parents):
    grouped1[p].append(c)
row1 = ""
for p, children in grouped1.items():
    row1 = row1 + f"{p}: {', '.join(children)}\n"

for c, p in zip(binary_2, binary2_parents):
    grouped2[p].append(c)
row2 = ""
for p, children in grouped2.items():
    row2 = row2 + f"{p}: {', '.join(children)}\n"

table_data = {
    "Morphine only": [row1],
    "Saline only": [row2]
}
table_df = pd.DataFrame(table_data)

html_table = table_df.to_html(index=False, escape=False).replace("\\n", "<br>")
display(HTML(html_table))

Morphine only,Saline only
"MY: AMB, AMBd, AMBv, LIN, NR, VI, XII, x, RO, AP, GR, ECU, Pa5P: PC5, V, RPO, SLC, SLD, KFHY: RCH, ME, TMd, AVP, AVPV, MEPO, OV, SCH, SFO, VLPO, VMPO, ASO, PVaTH: IntG, PR, VPLpc, VPMpcMB: LT, MA3, Su3, PN, MEV, CLI, IPA, IPDMPAL: BACSTR: BAIsocortex: ACAd6b, ACAv6b, AId6b, AIp6b, AIv6b, AUDd6b, ORBl6b, ORBm6b, PL6b, SSp-ll6b, SSp-tr6b, SSp-ul6b, SSp-un6bOLF: NLOT1",MY: RPAMB: SCO


#### Female morphine vs female saline

In [99]:
morphine_dfs_female = []
saline_dfs_female = []
for df_path in all_dfs:
    df_path_dict = density_dfs[df_path]
    density_df = df_path_dict['df']
    treatment = df_path_dict['treatment']
    sex = df_path_dict['sex']
    if treatment == "morphine" and sex == 'f':
        morphine_dfs_female.append(density_df)
    elif treatment == "saline" and sex == 'f':
        saline_dfs_female.append(density_df)

# create merged dfs (with repeating structures) - 1 for morphine, 1 for saline
morphine_dfs_female = pd.concat(morphine_dfs_female)
assert morphine_dfs_female.shape[0] == 840 * 6

saline_dfs_female = pd.concat(saline_dfs_female)
assert saline_dfs_female.shape[0] == 840 * 6

morphine_dfs_female.sort_values(by='Structure', inplace=True)
saline_dfs_female.sort_values(by='Structure', inplace=True)

morphine_dfs_female.reset_index(drop=True, inplace=True)
saline_dfs_female.reset_index(drop=True, inplace=True)

morphine_average_cell_count_df = morphine_dfs_female.groupby('Structure')['cell_count'].mean().reset_index()
saline_average_cell_count_df = saline_dfs_female.groupby('Structure')['cell_count'].mean().reset_index()

merged_df = pd.merge(morphine_average_cell_count_df, saline_average_cell_count_df, on='Structure', suffixes=('_morphine', '_saline'))
merged_df = merged_df[['Structure', 'cell_count_morphine', 'cell_count_saline']]

get_acronym = lambda x: x.split(">")[-1]
merged_df['acronym'] = list(map(get_acronym, merged_df['Structure'].to_list()))

def is_grey(acronym):
    parents = atlas.get_structure_ancestors(acronym)
    return "grey" in parents

merged_df['is_grey'] = list(map(is_grey, merged_df['acronym'].to_list()))
merged_df = merged_df[merged_df['is_grey'] == True]

binary_1 = []  # morphine only (red)
binary_2 = []  # saline only (blue)
binary_3 = []  # both are zero (grey)

for ind, row in merged_df.iterrows():
    if (row['cell_count_saline'] < 1) and (row['cell_count_morphine'] >= 1):
        binary_1.append(row['acronym'])
    elif (row['cell_count_morphine'] < 1) and (row['cell_count_saline'] >= 1):
        binary_2.append(row['acronym'])
    elif (row['cell_count_saline'] < 1) and (row['cell_count_morphine'] < 1):
        binary_3.append(row['acronym'])
# print("Morphine only", binary_1)
# print("Saline only", binary_2)
# print("No cells in either", binary_3)

binary1_parents = list(map(get_major_parent, binary_1))
binary2_parents = list(map(get_major_parent, binary_2))

grouped1 = defaultdict(list)
grouped2 = defaultdict(list)

for c, p in zip(binary_1, binary1_parents):
    grouped1[p].append(c)
row1 = ""
for p, children in grouped1.items():
    row1 = row1 + f"{p}: {', '.join(children)}\n"

for c, p in zip(binary_2, binary2_parents):
    grouped2[p].append(c)
row2 = ""
for p, children in grouped2.items():
    row2 = row2 + f"{p}: {', '.join(children)}\n"

table_data = {
    "Morphine only": [row1],
    "Saline only": [row2]
}
table_df = pd.DataFrame(table_data)

html_table = table_df.to_html(index=False, escape=False).replace("\\n", "<br>")
display(HTML(html_table))

Morphine only,Saline only
"MY: ICB, PAS, PRP, AP, GR, ECUP: P5, SG, V, SLCHY: ME, SFOTH: RHMB: III, LT, INC, Su3, IPA, IPDL, IPDM, IPRLIsocortex: GU6b, ILA6b, MOs6b, ORBvl6b, VISa1, VISrl1, VISrl6b, SSp-ll6b, SSp-m6b, SSp-n6b, SSp-tr1, SSp-tr6b, SSp-un6b, VISal6b","MY: AMB, AMBv, PPY, RPAP: PC5TH: LGd-ipMB: PBG, SCOCB: LINGSTR: SH, BAIsocortex: VISam6b"


#### Morphine 1h vs morphine 4h

In [100]:
morphine_dfs_1h = []
for df_path in all_dfs:
    df_path_dict = density_dfs[df_path]
    density_df = df_path_dict['df']
    treatment = df_path_dict['treatment']
    time_point = df_path_dict['time_point']
    if treatment == "morphine" and time_point == 1:
        morphine_dfs_1h.append(density_df)

morphine_dfs_1h = pd.concat(morphine_dfs_1h)
assert morphine_dfs_1h.shape[0] == 840 * 8

morphine_dfs_4h = []
for df_path in all_dfs:
    cells_df = pd.read_csv(df_path, sep="\t")
    density_df = pd.read_csv(df_path.replace(".csv", "all_structures_density.csv"))
    treatment = cells_df['treatment'].unique()[0]
    time_point = cells_df['time_point'].unique()[0]
    if treatment == "morphine" and time_point == 4:
        morphine_dfs_4h.append(density_df)

morphine_dfs_4h = pd.concat(morphine_dfs_4h)
assert morphine_dfs_4h.shape[0] == 840 * 7

morphine_1h_average_cell_count_df = morphine_dfs_1h.groupby('Structure')['cell_count'].mean().reset_index()
morphine_4h_average_cell_count_df = morphine_dfs_4h.groupby('Structure')['cell_count'].mean().reset_index()

merged_df = pd.merge(morphine_1h_average_cell_count_df, morphine_4h_average_cell_count_df, on='Structure', suffixes=('_morphine_1h', '_morphine_4h'))
merged_df = merged_df[['Structure', 'cell_count_morphine_1h', 'cell_count_morphine_4h']]

get_acronym = lambda x: x.split(">")[-1]
merged_df['acronym'] = list(map(get_acronym, merged_df['Structure'].to_list()))

def is_grey(acronym):
    parents = atlas.get_structure_ancestors(acronym)
    return "grey" in parents

merged_df['is_grey'] = list(map(is_grey, merged_df['acronym'].to_list()))
merged_df = merged_df[merged_df['is_grey'] == True]

binary_1 = []  # 4h only (red)
binary_2 = []  # 1h only (blue)
binary_3 = []  # both are zero (black)

for ind, row in merged_df.iterrows():
    if (row['cell_count_morphine_1h'] < 1) and (row['cell_count_morphine_4h'] >= 1):
        binary_1.append(row['acronym'])
    elif (row['cell_count_morphine_4h'] < 1) and (row['cell_count_morphine_1h'] >= 1):
        binary_2.append(row['acronym'])
    elif (row['cell_count_morphine_1h'] < 1) and (row['cell_count_morphine_4h'] < 1):
        binary_3.append(row['acronym'])
# print("4h only", binary_1)
# print("1h only", binary_2)
# print("No cells in either", binary_3)

binary1_parents = list(map(get_major_parent, binary_1))
binary2_parents = list(map(get_major_parent, binary_2))

grouped1 = defaultdict(list)
grouped2 = defaultdict(list)

for c, p in zip(binary_1, binary1_parents):
    grouped1[p].append(c)
row1 = ""
for p, children in grouped1.items():
    row1 = row1 + f"{p}: {', '.join(children)}\n"

for c, p in zip(binary_2, binary2_parents):
    grouped2[p].append(c)
row2 = ""
for p, children in grouped2.items():
    row2 = row2 + f"{p}: {', '.join(children)}\n"

table_data = {
    "4h only": [row1],
    "1h only": [row2]
}
table_df = pd.DataFrame(table_data)

html_table = table_df.to_html(index=False, escape=False).replace("\\n", "<br>")
display(HTML(html_table))

4h only,1h only
"MY: PAS, NR, VI, RO, APHY: OV, SFO, VMPOTH: RHSTR: BAIsocortex: ORBm6b","MY: AMBd, ICB, LIN, y, Pa5TH: IGL, IntG, LGd-ip, LGd-shIsocortex: AIv6b, VISam6b, VISpm6b"


#### Morphine male vs morphine female

In [101]:
morphine_dfs_male = []
morphine_dfs_female = []
for df_path in all_dfs:
    df_path_dict = density_dfs[df_path]
    density_df = df_path_dict['df']
    treatment = df_path_dict['treatment']
    sex = df_path_dict['sex']
    if treatment == "morphine" and sex == 'm':
        morphine_dfs_male.append(density_df)
    elif treatment == "morphine" and sex == 'f':
        morphine_dfs_female.append(density_df)
    else:
        pass

# create merged dfs (with repeating structures) - 1 for morphine, 1 for saline
morphine_dfs_male = pd.concat(morphine_dfs_male)
assert morphine_dfs_male.shape[0] == 840 * 9

morphine_dfs_female = pd.concat(morphine_dfs_female)
assert morphine_dfs_female.shape[0] == 840 * 6

morphine_dfs_male.sort_values(by='Structure', inplace=True)
morphine_dfs_female.sort_values(by='Structure', inplace=True)

morphine_dfs_male.reset_index(drop=True, inplace=True)
morphine_dfs_female.reset_index(drop=True, inplace=True)

morphine_male_average_cell_count_df = morphine_dfs_male.groupby('Structure')['cell_count'].mean().reset_index()
morphine_female_average_cell_count_df = morphine_dfs_female.groupby('Structure')['cell_count'].mean().reset_index()

merged_df = pd.merge(morphine_male_average_cell_count_df, morphine_female_average_cell_count_df, on='Structure', suffixes=('_morphine_male', '_morphine_female'))
merged_df = merged_df[['Structure', 'cell_count_morphine_male', 'cell_count_morphine_female']]

get_acronym = lambda x: x.split(">")[-1]
merged_df['acronym'] = list(map(get_acronym, merged_df['Structure'].to_list()))

def is_grey(acronym):
    parents = atlas.get_structure_ancestors(acronym)
    return "grey" in parents

merged_df['is_grey'] = list(map(is_grey, merged_df['acronym'].to_list()))
merged_df = merged_df[merged_df['is_grey'] == True]

binary_1 = []  # male only (red)
binary_2 = []  # female only (blue)
binary_3 = []  # both are zero (black)

for ind, row in merged_df.iterrows():
    if (row['cell_count_morphine_female'] < 1) and (row['cell_count_morphine_male'] >= 1):
        binary_1.append(row['acronym'])
    elif (row['cell_count_morphine_male'] < 1) and (row['cell_count_morphine_female'] >= 1):
        binary_2.append(row['acronym'])
    elif (row['cell_count_morphine_male'] < 1) and (row['cell_count_morphine_female'] < 1):
        binary_3.append(row['acronym'])
# print("Male only", binary_1)
# print("Female only", binary_2)
# print("No cells in either", binary_3)

binary1_parents = list(map(get_major_parent, binary_1))
binary2_parents = list(map(get_major_parent, binary_2))

grouped1 = defaultdict(list)
grouped2 = defaultdict(list)

for c, p in zip(binary_1, binary1_parents):
    grouped1[p].append(c)
row1 = ""
for p, children in grouped1.items():
    row1 = row1 + f"{p}: {', '.join(children)}\n"

for c, p in zip(binary_2, binary2_parents):
    grouped2[p].append(c)
row2 = ""
for p, children in grouped2.items():
    row2 = row2 + f"{p}: {', '.join(children)}\n"

table_data = {
    "Male only": [row1],
    "Female only": [row2]
}
table_df = pd.DataFrame(table_data)

html_table = table_df.to_html(index=False, escape=False).replace("\\n", "<br>")
display(HTML(html_table))

Male only,Female only
"MY: AMB, AMBd, AMBv, LRNp, NR, PPY, VI, y, RO, Pa5P: PC5TH: IGL, IntG, LGd-ip, LGd-shMB: DT, PBGCB: LINGSTR: SH, BAIsocortex: AIv6b, ORBm6b, RSPv6b, VISam6b, VISpm6b",TH: RH


# Smallest groups

### Table S8 - significance vs all leaf structures with gray matter <a id="tables8"/>

In [62]:
def get_significance_marker(p_value):
    """
    Returns statistical significance markers based on p-value.

    Parameters:
    p_value (float): The p-value to evaluate.

    Returns:
    str: Significance marker ('ns', '*', '**', '***', '****').
    """
    if p_value < 0.0001:
        return '****'  # Highly significant
    elif p_value < 0.001:
        return '***'   # Very significant
    elif p_value < 0.01:
        return '**'    # Significant
    elif p_value < 0.05:
        return '*'     # Marginally significant
    else:
        return ''    # Not significant

In [63]:
tree_leaves = [x.tag.split()[0] for x in tree.leaves()]
grey_tree_leaves = []
for leaf in tree_leaves:
    try:
        if is_grey(leaf):
            grey_tree_leaves.append(leaf)
    except KeyError:
        pass


morphine_dfs_male_1h = []
saline_dfs_male_1h = []
for df_path in all_dfs:
    df_path_dict = density_dfs[df_path]
    density_df = df_path_dict['df']
    treatment = df_path_dict['treatment']
    time_point = df_path_dict['time_point']
    sex = df_path_dict['sex']
    if treatment == "morphine" and time_point == 1 and sex == 'm':
        morphine_dfs_male_1h.append(density_df)
    elif treatment == "saline" and time_point == 1 and sex == 'm':
        saline_dfs_male_1h.append(density_df)

# create merged dfs (with repeating structures) - 1 for morphine, 1 for saline
morphine_dfs_male_1h = pd.concat(morphine_dfs_male_1h)
assert morphine_dfs_male_1h.shape[0] == 840 * 5

saline_dfs_male_1h = pd.concat(saline_dfs_male_1h)
assert saline_dfs_male_1h.shape[0] == 840 * 3

morphine_dfs_male_1h.sort_values(by='Structure', inplace=True)
saline_dfs_male_1h.sort_values(by='Structure', inplace=True)

morphine_dfs_male_1h.reset_index(drop=True, inplace=True)
saline_dfs_male_1h.reset_index(drop=True, inplace=True)

morphine_dfs_male_1h['acronym'] = list(map(get_acronym, morphine_dfs_male_1h['Structure'].to_list()))
saline_dfs_male_1h['acronym'] = list(map(get_acronym, saline_dfs_male_1h['Structure'].to_list()))


morphine_dfs_female_1h = []
saline_dfs_female_1h = []
for df_path in all_dfs:
    df_path_dict = density_dfs[df_path]
    density_df = df_path_dict['df']
    treatment = df_path_dict['treatment']
    time_point = df_path_dict['time_point']
    sex = df_path_dict['sex']
    if treatment == "morphine" and time_point == 1 and sex == 'f':
        morphine_dfs_female_1h.append(density_df)
    elif treatment == "saline" and time_point == 1 and sex == 'f':
        saline_dfs_female_1h.append(density_df)

# create merged dfs (with repeating structures) - 1 for morphine, 1 for saline
morphine_dfs_female_1h = pd.concat(morphine_dfs_female_1h)
assert morphine_dfs_female_1h.shape[0] == 840 * 3

saline_dfs_female_1h = pd.concat(saline_dfs_female_1h)
assert saline_dfs_female_1h.shape[0] == 840 * 2

morphine_dfs_female_1h.sort_values(by='Structure', inplace=True)
saline_dfs_female_1h.sort_values(by='Structure', inplace=True)

morphine_dfs_female_1h.reset_index(drop=True, inplace=True)
saline_dfs_female_1h.reset_index(drop=True, inplace=True)

morphine_dfs_female_1h['acronym'] = list(map(get_acronym, morphine_dfs_female_1h['Structure'].to_list()))
saline_dfs_female_1h['acronym'] = list(map(get_acronym, saline_dfs_female_1h['Structure'].to_list()))

morphine_dfs_male_4h = []
saline_dfs_male_4h = []
for df_path in all_dfs:
    df_path_dict = density_dfs[df_path]
    density_df = df_path_dict['df']
    treatment = df_path_dict['treatment']
    time_point = df_path_dict['time_point']
    sex = df_path_dict['sex']
    if treatment == "morphine" and time_point == 4 and sex == 'm':
        morphine_dfs_male_4h.append(density_df)
    elif treatment == "saline" and time_point == 4 and sex == 'm':
        saline_dfs_male_4h.append(density_df)

# create merged dfs (with repeating structures) - 1 for morphine, 1 for saline
morphine_dfs_male_4h = pd.concat(morphine_dfs_male_4h)
assert morphine_dfs_male_4h.shape[0] == 840 * 4

saline_dfs_male_4h = pd.concat(saline_dfs_male_4h)
assert saline_dfs_male_4h.shape[0] == 840 * 3

morphine_dfs_male_4h.sort_values(by='Structure', inplace=True)
saline_dfs_male_4h.sort_values(by='Structure', inplace=True)

morphine_dfs_male_4h.reset_index(drop=True, inplace=True)
saline_dfs_male_4h.reset_index(drop=True, inplace=True)

morphine_dfs_male_4h['acronym'] = list(map(get_acronym, morphine_dfs_male_4h['Structure'].to_list()))
saline_dfs_male_4h['acronym'] = list(map(get_acronym, saline_dfs_male_4h['Structure'].to_list()))


morphine_dfs_female_4h = []
saline_dfs_female_4h = []
for df_path in all_dfs:
    df_path_dict = density_dfs[df_path]
    density_df = df_path_dict['df']
    treatment = df_path_dict['treatment']
    time_point = df_path_dict['time_point']
    sex = df_path_dict['sex']
    if treatment == "morphine" and time_point == 4 and sex == 'f':
        morphine_dfs_female_4h.append(density_df)
    elif treatment == "saline" and time_point == 4 and sex == 'f':
        saline_dfs_female_4h.append(density_df)

# create merged dfs (with repeating structures) - 1 for morphine, 1 for saline
morphine_dfs_female_4h = pd.concat(morphine_dfs_female_4h)
assert morphine_dfs_female_4h.shape[0] == 840 * 3

saline_dfs_female_4h = pd.concat(saline_dfs_female_4h)
assert saline_dfs_female_4h.shape[0] == 840 * 4

morphine_dfs_female_4h.sort_values(by='Structure', inplace=True)
saline_dfs_female_4h.sort_values(by='Structure', inplace=True)

morphine_dfs_female_4h.reset_index(drop=True, inplace=True)
saline_dfs_female_4h.reset_index(drop=True, inplace=True)

morphine_dfs_female_4h['acronym'] = list(map(get_acronym, morphine_dfs_female_4h['Structure'].to_list()))
saline_dfs_female_4h['acronym'] = list(map(get_acronym, saline_dfs_female_4h['Structure'].to_list()))


saline_1h_male_average_density_df = saline_dfs_male_1h.groupby('Structure')['density'].mean().reset_index()
saline_1h_female_average_density_df = saline_dfs_female_1h.groupby('Structure')['density'].mean().reset_index()
saline_4h_male_average_density_df = saline_dfs_male_4h.groupby('Structure')['density'].mean().reset_index()
saline_4h_female_average_density_df = saline_dfs_female_4h.groupby('Structure')['density'].mean().reset_index()

morphine_1h_male_average_density_df = morphine_dfs_male_1h.groupby('Structure')['density'].mean().reset_index()
morphine_1h_female_average_density_df = morphine_dfs_female_1h.groupby('Structure')['density'].mean().reset_index()
morphine_4h_male_average_density_df = morphine_dfs_male_4h.groupby('Structure')['density'].mean().reset_index()
morphine_4h_female_average_density_df = morphine_dfs_female_4h.groupby('Structure')['density'].mean().reset_index()


morphine_average_density_df = morphine_dfs.groupby('Structure')['density'].mean().reset_index()
saline_average_density_df = saline_dfs.groupby('Structure')['density'].mean().reset_index()
morphine_average_density_df['acronym'] = list(map(get_acronym, morphine_average_density_df['Structure'].to_list()))
saline_average_density_df['acronym'] = list(map(get_acronym, saline_average_density_df['Structure'].to_list()))
morphine_1h_average_density_df['acronym'] = list(map(get_acronym, morphine_1h_average_density_df['Structure'].to_list()))
morphine_4h_average_density_df['acronym'] = list(map(get_acronym, morphine_4h_average_density_df['Structure'].to_list()))
morphine_male_average_density_df['acronym'] = list(map(get_acronym, morphine_male_average_density_df['Structure'].to_list()))
morphine_female_average_density_df['acronym'] = list(map(get_acronym, morphine_female_average_density_df['Structure'].to_list()))
saline_1h_average_density_df['acronym'] = list(map(get_acronym, saline_1h_average_density_df['Structure'].to_list()))
saline_4h_average_density_df['acronym'] = list(map(get_acronym, saline_4h_average_density_df['Structure'].to_list()))
saline_male_average_density_df['acronym'] = list(map(get_acronym, saline_male_average_density_df['Structure'].to_list()))
saline_female_average_density_df['acronym'] = list(map(get_acronym, saline_female_average_density_df['Structure'].to_list()))


saline_1h_male_average_density_df['acronym'] = list(map(get_acronym, saline_1h_male_average_density_df['Structure'].to_list()))
saline_1h_female_average_density_df['acronym'] = list(map(get_acronym, saline_1h_female_average_density_df['Structure'].to_list()))
saline_4h_male_average_density_df['acronym'] = list(map(get_acronym, saline_4h_male_average_density_df['Structure'].to_list()))
saline_4h_female_average_density_df['acronym'] = list(map(get_acronym, saline_4h_female_average_density_df['Structure'].to_list()))

morphine_1h_male_average_density_df['acronym'] = list(map(get_acronym, morphine_1h_male_average_density_df['Structure'].to_list()))
morphine_1h_female_average_density_df['acronym'] = list(map(get_acronym, morphine_1h_female_average_density_df['Structure'].to_list()))
morphine_4h_male_average_density_df['acronym'] = list(map(get_acronym, morphine_4h_male_average_density_df['Structure'].to_list()))
morphine_4h_female_average_density_df['acronym'] = list(map(get_acronym, morphine_4h_female_average_density_df['Structure'].to_list()))


merged_df_1h_male = pd.merge(morphine_1h_male_average_density_df, saline_1h_male_average_density_df, on='Structure', suffixes=('_morphine', '_saline'))
merged_df_1h_male = merged_df_1h_male[merged_df_1h_male['density_saline'] > 0]   # TODO: how do we deal with binary effects?
merged_df_1h_male['density_ratio'] = merged_df_1h_male['density_morphine'] / merged_df_1h_male['density_saline']

density_ratio_df_1h_male = merged_df_1h_male[['Structure', 'density_ratio']]

# calculate p-values for every structure
p_values = []
significance = []
for structure in density_ratio_df_1h_male['Structure'].to_list():
    morphine_dataset = morphine_dfs_male_1h[morphine_dfs_male_1h['Structure'] == structure]['density'].to_list()
#     assert len(morphine_dataset) == 8
    saline_dataset = saline_dfs_male_1h[saline_dfs_male_1h['Structure'] == structure]['density'].to_list()
#     assert len(saline_dataset) == 5
    p_val = get_p_value(morphine_dataset, saline_dataset)
    signif = p_val <= significance_threshold
    p_values.append(p_val)
    significance.append(signif)

density_ratio_df_1h_male['p_value'] = p_values
density_ratio_df_1h_male['significant'] = significance

density_ratio_df_1h_male['acronym'] = list(map(get_acronym, density_ratio_df_1h_male['Structure'].to_list()))

merged_df_1h_female = pd.merge(morphine_1h_female_average_density_df, saline_1h_female_average_density_df, on='Structure', suffixes=('_morphine', '_saline'))
merged_df_1h_female = merged_df_1h_female[merged_df_1h_female['density_saline'] > 0]   # TODO: how do we deal with binary effects?
merged_df_1h_female['density_ratio'] = merged_df_1h_female['density_morphine'] / merged_df_1h_female['density_saline']

density_ratio_df_1h_female = merged_df_1h_female[['Structure', 'density_ratio']]

# calculate p-values for every structure
p_values = []
significance = []
for structure in density_ratio_df_1h_female['Structure'].to_list():
    morphine_dataset = morphine_dfs_female_1h[morphine_dfs_female_1h['Structure'] == structure]['density'].to_list()
#     assert len(morphine_dataset) == 8
    saline_dataset = saline_dfs_female_1h[saline_dfs_female_1h['Structure'] == structure]['density'].to_list()
#     assert len(saline_dataset) == 5
    p_val = get_p_value(morphine_dataset, saline_dataset)
    signif = p_val <= significance_threshold
    p_values.append(p_val)
    significance.append(signif)

density_ratio_df_1h_female['p_value'] = p_values
density_ratio_df_1h_female['significant'] = significance

density_ratio_df_1h_female['acronym'] = list(map(get_acronym, density_ratio_df_1h_female['Structure'].to_list()))



merged_df_4h_male = pd.merge(morphine_4h_male_average_density_df, saline_4h_male_average_density_df, on='Structure', suffixes=('_morphine', '_saline'))
merged_df_4h_male = merged_df_4h_male[merged_df_4h_male['density_saline'] > 0]   # TODO: how do we deal with binary effects?
merged_df_4h_male['density_ratio'] = merged_df_4h_male['density_morphine'] / merged_df_4h_male['density_saline']

density_ratio_df_4h_male = merged_df_4h_male[['Structure', 'density_ratio']]

# calculate p-values for every structure
p_values = []
significance = []
for structure in density_ratio_df_4h_male['Structure'].to_list():
    morphine_dataset = morphine_dfs_male_4h[morphine_dfs_male_4h['Structure'] == structure]['density'].to_list()
#     assert len(morphine_dataset) == 8
    saline_dataset = saline_dfs_male_4h[saline_dfs_male_4h['Structure'] == structure]['density'].to_list()
#     assert len(saline_dataset) == 5
    p_val = get_p_value(morphine_dataset, saline_dataset)
    signif = p_val <= significance_threshold
    p_values.append(p_val)
    significance.append(signif)

density_ratio_df_4h_male['p_value'] = p_values
density_ratio_df_4h_male['significant'] = significance

density_ratio_df_4h_male['acronym'] = list(map(get_acronym, density_ratio_df_4h_male['Structure'].to_list()))



merged_df_4h_female = pd.merge(morphine_4h_female_average_density_df, saline_4h_female_average_density_df, on='Structure', suffixes=('_morphine', '_saline'))
merged_df_4h_female = merged_df_4h_female[merged_df_4h_female['density_saline'] > 0]   # TODO: how do we deal with binary effects?
merged_df_4h_female['density_ratio'] = merged_df_4h_female['density_morphine'] / merged_df_4h_female['density_saline']

density_ratio_df_4h_female = merged_df_4h_female[['Structure', 'density_ratio']]

# calculate p-values for every structure
p_values = []
significance = []
for structure in density_ratio_df_4h_female['Structure'].to_list():
    morphine_dataset = morphine_dfs_female_4h[morphine_dfs_female_4h['Structure'] == structure]['density'].to_list()
#     assert len(morphine_dataset) == 8
    saline_dataset = saline_dfs_female_4h[saline_dfs_female_4h['Structure'] == structure]['density'].to_list()
#     assert len(saline_dataset) == 5
    p_val = get_p_value(morphine_dataset, saline_dataset)
    signif = p_val <= significance_threshold
    p_values.append(p_val)
    significance.append(signif)

density_ratio_df_4h_female['p_value'] = p_values
density_ratio_df_4h_female['significant'] = significance

density_ratio_df_4h_female['acronym'] = list(map(get_acronym, density_ratio_df_4h_female['Structure'].to_list()))

In [64]:
significance_table = pd.DataFrame()
significance_table['acronym'] = grey_tree_leaves


all_morphine_vs_all_saline_significance = []
for leaf in grey_tree_leaves:
    try:
        marker = get_significance_marker(density_ratio_df[density_ratio_df['acronym'] == leaf]['p_value'].values[0])
    except IndexError:
        marker = 'bin'
    all_morphine_vs_all_saline_significance.append(marker)

significance_table['all_M_vs_S'] = all_morphine_vs_all_saline_significance


male_morphine_vs_male_saline_significance = []
for leaf in grey_tree_leaves:
    try:
        marker = get_significance_marker(density_ratio_df_male[density_ratio_df_male['acronym'] == leaf]['p_value'].values[0])
    except IndexError:
        marker = 'bin'
    male_morphine_vs_male_saline_significance.append(marker)

significance_table['male_M_vs_S'] = male_morphine_vs_male_saline_significance


female_morphine_vs_female_saline_significance = []
for leaf in grey_tree_leaves:
    try:
        marker = get_significance_marker(density_ratio_df_female[density_ratio_df_female['acronym'] == leaf]['p_value'].values[0])
    except IndexError:
        marker = 'bin'
    female_morphine_vs_female_saline_significance.append(marker)

significance_table['female_M_vs_S'] = female_morphine_vs_female_saline_significance


one_morphine_vs_one_saline_significance = []
for leaf in grey_tree_leaves:
    try:
        marker = get_significance_marker(density_ratio_df_1h[density_ratio_df_1h['acronym'] == leaf]['p_value'].values[0])
    except IndexError:
        marker = 'bin'
    one_morphine_vs_one_saline_significance.append(marker)

significance_table['1h_M_vs_S'] = one_morphine_vs_one_saline_significance


four_morphine_vs_four_saline_significance = []
for leaf in grey_tree_leaves:
    try:
        marker = get_significance_marker(density_ratio_df_4h[density_ratio_df_4h['acronym'] == leaf]['p_value'].values[0])
    except IndexError:
        marker = 'bin'
    four_morphine_vs_four_saline_significance.append(marker)

significance_table['4h_M_vs_S'] = four_morphine_vs_four_saline_significance

one_male_morphine_vs_one_male_saline_significance = []
for leaf in grey_tree_leaves:
    try:
        marker = get_significance_marker(density_ratio_df_1h_male[density_ratio_df_1h_male['acronym'] == leaf]['p_value'].values[0])
    except IndexError:
        marker = 'bin'
    one_male_morphine_vs_one_male_saline_significance.append(marker)

significance_table['1h_male_M_vs_S'] = one_male_morphine_vs_one_male_saline_significance

one_female_morphine_vs_one_female_saline_significance = []
for leaf in grey_tree_leaves:
    try:
        marker = get_significance_marker(density_ratio_df_1h_female[density_ratio_df_1h_female['acronym'] == leaf]['p_value'].values[0])
    except IndexError:
        marker = 'bin'
    one_female_morphine_vs_one_female_saline_significance.append(marker)

significance_table['1h_female_M_vs_S'] = one_female_morphine_vs_one_female_saline_significance

four_male_morphine_vs_four_male_saline_significance = []
for leaf in grey_tree_leaves:
    try:
        marker = get_significance_marker(density_ratio_df_4h_male[density_ratio_df_4h_male['acronym'] == leaf]['p_value'].values[0])
    except IndexError:
        marker = 'bin'
    four_male_morphine_vs_four_male_saline_significance.append(marker)

significance_table['4h_male_M_vs_S'] = four_male_morphine_vs_four_male_saline_significance

four_female_morphine_vs_four_female_saline_significance = []
for leaf in grey_tree_leaves:
    try:
        marker = get_significance_marker(density_ratio_df_4h_female[density_ratio_df_4h_female['acronym'] == leaf]['p_value'].values[0])
    except IndexError:
        marker = 'bin'
    four_female_morphine_vs_four_female_saline_significance.append(marker)

significance_table['4h_female_M_vs_S'] = four_female_morphine_vs_four_female_saline_significance

# significance_table.to_csv('significance_vs_struct_and_exper_extended_reproduce.csv')

In [65]:
display(HTML(significance_table.to_html()))

,acronym,all_M_vs_S,male_M_vs_S,female_M_vs_S,1h_M_vs_S,4h_M_vs_S,1h_male_M_vs_S,1h_female_M_vs_S,4h_male_M_vs_S,4h_female_M_vs_S
0,FRP1,,,,,,,*,,
1,FRP2/3,,,,,,,,,
2,FRP5,,,,,,,,,
3,FRP6a,,*,,,,,,,
4,FRP6b,bin,bin,bin,bin,bin,bin,bin,bin,bin
5,MOp1,,,,,,,,,
6,MOp2/3,,,,,,,,,
7,MOp5,,,,,,,,,
8,MOp6a,,,,,,,,,
9,MOp6b,,,,,,,,bin,


In [66]:
end_time = datetime.now()
print("The whole notebook took", end_time - start_time)

The whole notebook took 0:01:14.333440
